In [37]:
from __future__ import print_function

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import logging
from time import time
import numpy as np

In [33]:
print("Loading 20 newsgroups dataset for categories:")
#print(categories)

dataset = fetch_20newsgroups(subset='train',shuffle=True, random_state=42)
test_dataset = fetch_20newsgroups(subset='test',shuffle=True,random_state=42)
print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))
print()

labels = dataset.target
true_k = np.unique(labels).shape[0]

Loading 20 newsgroups dataset for categories:
11314 documents
20 categories



In [27]:
opt_features = 250
opt_hash = False
opt_tfidf = True
opt_components = 200
opt_minibatch = True

Two feature extraction methods can be used:

    TfidfVectorizer uses a in-memory vocabulary (a python dict) to map the most frequent words to features indices and hence compute a word occurrence frequency (sparse) matrix. The word frequencies are then reweighted using the Inverse Document Frequency (IDF) vector collected feature-wise over the corpus.

    HashingVectorizer hashes word occurrences to a fixed dimensional space, possibly with collisions. The word count vectors are then normalized to each have l2-norm equal to one (projected to the euclidean unit-ball) which seems to be important for k-means to work in high dimensional space.

    HashingVectorizer does not provide IDF weighting as this is a stateless model (the fit method does nothing). When IDF weighting is needed it can be added by pipelining its output to a TfidfTransformer instance.



In [34]:
print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()
# Perform an IDF normalization on the output of HashingVectorizer
if opt_hash:
    hasher = HashingVectorizer(n_features = opt_features,\
            stop_words='english', non_negative=True, norm=None, binary=False)
    if opt_tfidf:
        vectorizer = make_pipeline(hasher, TfidfTransformer())

# Perform hashing with no IDF normalisation on the output of HashingVectorizer
    else:
        vectorizer = HashingVectorizer(n_features=opt_features,stop_words='english',\
            non_negative=False, norm='l2',binary=False)

else:
    #Perform only IDF normalisation
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opt_features,min_df=2, stop_words='english',
                                 use_idf=opt_tfidf)
X = vectorizer.fit_transform(dataset.data)
X_test = vectorizer.fit_transform(test_dataset.data)
print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

Extracting features from the training dataset using a sparse vectorizer
done in 6.984908s
n_samples: 11314, n_features: 250



In [35]:
if opt_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(opt_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)
    X_test = lsa.fit_transform(X_test)
    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()

Performing dimensionality reduction using LSA
done in 2.828565s
Explained variance of the SVD step: 90%



k-means (and minibatch k-means) are very sensitive to feature scaling and that in this case the IDF weighting helps improve the quality of the clustering by quite a lot as measured against the "ground truth" provided by the class label assignments of the 20 newsgroups dataset.

This improvement is not visible in the Silhouette Coefficient which is small for both as this measure seem to suffer from the phenomenon called "Concentration of Measure" or "Curse of Dimensionality" for high dimensional datasets such as text data. Other measures such as V-measure and Adjusted Rand Index are information theoretic based evaluation scores: as they are only based on cluster assignments rather than distances, hence not affected by the curse of dimensionality.

In [30]:
if opt_minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

print()

Clustering sparse data with MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=1000, max_iter=100, max_no_improvement=10, n_clusters=20,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)
done in 0.341s

Homogeneity: 0.203
Completeness: 0.216
V-measure: 0.209
Adjusted Rand-Index: 0.078
Silhouette Coefficient: 0.045



In [32]:
if opt_components:
    original_space_centroids = svd.inverse_transform(km.cluster_centers_)
    order_centroids = original_space_centroids.argsort()[:, ::-1]
else:
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()
print()

Cluster 0: university host posting nntp thanks mail distribution washington usa reply
Cluster 1: apple mac com scsi know use article university problem like
Cluster 2: game don good like just use article time know university
Cluster 3: god jesus people believe christian does say think life don
Cluster 4: uk ac university article host posting nntp com just know
Cluster 5: people don think like just com said did know article
Cluster 6: com article posting nntp host reply sun distribution ibm systems
Cluster 7: ca university article com posting like host nntp don just
Cluster 8: nasa gov space article com center posting host research nntp
Cluster 9: state drive university scsi hard new nntp host posting article
Cluster 10: windows file dos files program use window com version help
Cluster 11: card 00 graphics university windows com pc thanks does know
Cluster 12: gun control people com don law like right think make
Cluster 13: team play game year ca university best new think article
Clust

In [36]:
result = km.predict(X_test)
np.mean (result==test_dataset.target)

0.043680297397769519